<a href="https://colab.research.google.com/github/KijoSal-dev/Bolt-OpenAI-Pinecone/blob/main/ChatOps_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# 2.3.1 Install
# pip install slack_bolt openai pinecone

import os
from slack_bolt import App
from openai import OpenAI
import pinecone

# 1. Initialize
# Note: Work in progress to address the APIs in Slack and OpenAI
app = App(token=os.environ["SLACK_BOT_TOKEN"],
          signing_secret=os.environ["SLACK_SIGNING_SECRET"])
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
pinecone.init(api_key=os.environ["PINECONE_API_KEY"],
              environment="us-west1-gcp")
index = pinecone.Index("code-snippets")

# 2. Helper to get embeddings
def embed(text):
    resp = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    return resp.data[0].embedding

# 3. Slash command handler
@app.command("/codesearch")
def handle_codesearch(ack, body, respond):
    ack()
    query = body["text"]
    q_emb  = embed(query)              # 3.1 embed user query
    res    = index.query(q_emb, top_k=3, include_metadata=True)
    # 3.2 Format top-3 results
    msg = "*Top 3 code matches:*\n"
    for match in res.matches:
        path = match.metadata["path"]
        snippet = match.metadata["snippet"]
        msg += f"> *{path}*\n```{snippet}```\n"
    respond(msg)

# 4. Start the app
if __name__ == "__main__":
    app.start(port=int(os.environ.get("PORT", 3000)))

KeyError: 'SLACK_BOT_TOKEN'

In [ ]:
!pip install slack_bolt openai pinecone